In [44]:
#import libraries

import random
import bisect
import numpy as np# Define input features:
from numpy import random
from IPython.display import clear_output

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt




# variable inputs
SEED = None
MAXSEEDSEQ = 100000
u1count = 0
counter = 0

vhrthin = pd.read_csv("thinningdata.csv")

In [45]:
##THINNING FOR INTERNAL NETWORK USERS [START,END]

## new node interal arc users [start,end]

#increase vhrthin.per hr by rate; and thin

inc_rate = 1.10



vhrthin['perhr'] = vhrthin.perhr*inc_rate

## thin based on new node

## generate ALL t* under maxlambda in 24 hr period and then thin
maxlambda = np.max(vhrthin.perhr)+.3

print("max lambda: ", maxlambda)

#generate first customer arrival
u1 = random.uniform(0,1)

lambda0 = vhrthin.loc[vhrthin.index == 0, 'perhr'].iloc[0]


cust1 = -1/lambda0*np.log(u1)

vtime = cust1


cust_total = []

cust_total.append(vtime)

while vtime < 24:
    u1 = random.uniform(0,1)
    vtime = vtime - (1/maxlambda)*np.log(u1)
    cust_total.append(vtime)
    if len(cust_total) > 1000: 
        break



cust_total.pop(-1) #pop inplace=True by default


cust_ftotal = []
customer_startinendin = []

ncust = 1

#thin arrival times here
for x in range(len(cust_total)):
    u2 = random.uniform(0,1)
    tlambda = np.floor(cust_total[x])
    tlambda = vhrthin.loc[vhrthin.hr == tlambda,'perhr'].iloc[0]
    if u2 <= tlambda / maxlambda:
        cust_ftotal.append(cust_total[x])
        customer_startinendin.append((ncust, cust_total[x]))
        ncust +=1

'''
##print list
cust_ftotal = pd.DataFrame(cust_ftotal)
cust_ftotal.columns = ["arrival_time"]
cust_ftotal['arrhour'] = cust_ftotal.arrival_time.apply(lambda x: np.floor(x))
cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))
(cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))).sum()
'''


customer_startinendin ## contains new customer arrivals in list of tuples [(cust #, arrival time in hrs)]

max lambda:  2.2842307692307693


24.100714557966672

'\n##print list\ncust_ftotal = pd.DataFrame(cust_ftotal)\ncust_ftotal.columns = ["arrival_time"]\ncust_ftotal[\'arrhour\'] = cust_ftotal.arrival_time.apply(lambda x: np.floor(x))\ncust_ftotal.groupby(\'arrhour\').agg(vcount = (\'arrhour\',\'count\'))\n(cust_ftotal.groupby(\'arrhour\').agg(vcount = (\'arrhour\',\'count\'))).sum()\n'

[(1, 6.909599202702908),
 (2, 9.058708053628884),
 (3, 9.474093817040579),
 (4, 9.542395884362485),
 (5, 10.646718936394187),
 (6, 11.099324009711774),
 (7, 11.525721909235683),
 (8, 12.396735809816066),
 (9, 12.718169486475709),
 (10, 13.249274965294662),
 (11, 14.806840459921743),
 (12, 15.382509485323924),
 (13, 16.004241206317754),
 (14, 16.28509609650372),
 (15, 17.704271889237788),
 (16, 19.008767766105613),
 (17, 19.05462695100638),
 (18, 19.23925019329417),
 (19, 19.722823846841283),
 (20, 20.596752649160727),
 (21, 21.5017457614969),
 (22, 22.122662570886867),
 (23, 23.784138664906564),
 (24, 23.84968650146487)]

In [46]:
## GET ARRIVAL TIMES FOR NON-CLOSED ARCS IE [START,END) AND (START,END]

dfoos = pd.read_csv("oos_newnode.csv")

dfoos

,hour,startinendout_lambda,loglambda,startoutendin_lambda
0,0.0,1.100000,0.000000,2.400000
1,1.0,1.100000,0.000000,1.874074
2,2.0,1.100000,0.000000,1.222222
3,3.0,1.100000,0.000000,1.100000
4,4.0,1.100000,0.000000,1.237500
5,5.0,7.107692,1.865867,2.946429
6,6.0,17.769231,2.782158,7.412308
7,7.0,41.884615,3.639608,9.070769
8,8.0,41.884615,3.639608,16.906154
9,9.0,20.984615,2.948479,8.969231


In [47]:
## arrival times for those that start IN network but end OUTSIDE network [start,end)


## generate ALL t* under maxlambda in 24 hr period and then thin
maxlambda = np.max(dfoos.startinendout_lambda)


#generate first customer arrival
u1 = random.uniform(0,1)

lambda0 = dfoos.loc[dfoos.index == 0, 'startinendout_lambda'].iloc[0]


cust1 = -1/lambda0*np.log(u1)

vtime = cust1


cust_total = []

cust_total.append(vtime)

while vtime < 24:
    u1 = random.uniform(0,1)
    vtime = vtime - (1/maxlambda)*np.log(u1)
    cust_total.append(vtime)
    if len(cust_total) > 1000: 
        break



cust_total.pop(-1) #pop inplace=True by default

len(cust_total)

cust_ftotal = []
customer_startinendout = []

ncust = 1

#thin arrival times here
for x in range(len(cust_total)):
    u2 = random.uniform(0,1)
    tlambda = np.floor(cust_total[x])
    tlambda = dfoos.loc[dfoos.hour == tlambda,'startinendout_lambda'].iloc[0]
    if u2 <= tlambda / maxlambda:
        cust_ftotal.append(cust_total[x])
        customer_startinendout.append((ncust, cust_total[x]))
        ncust +=1

cust_ftotal = pd.DataFrame(cust_ftotal)
cust_ftotal.columns = ["arrival_time"]
cust_ftotal['arrhour'] = cust_ftotal.arrival_time.apply(lambda x: np.floor(x))

##print list
cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))
(cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))).sum()

customer_startinendout

24.008315424717157

1000

,vcount
arrhour,
0.0,1
1.0,2
2.0,2
4.0,2
5.0,9
6.0,17
7.0,37
8.0,39
9.0,25


vcount    344
dtype: int64

[(1, 0.2502246559853615),
 (2, 1.1845350817314202),
 (3, 1.8444294690823342),
 (4, 2.4519403584267176),
 (5, 2.547653758974186),
 (6, 4.303500207685058),
 (7, 4.498790089658464),
 (8, 5.036809562407914),
 (9, 5.049319277394381),
 (10, 5.09131029601956),
 (11, 5.224298185871785),
 (12, 5.36706949365861),
 (13, 5.568928915976376),
 (14, 5.636799269139683),
 (15, 5.807327387450063),
 (16, 5.935478440127755),
 (17, 6.052060015008623),
 (18, 6.151153244352603),
 (19, 6.169249160180376),
 (20, 6.216131382733089),
 (21, 6.238239720829),
 (22, 6.315917338015786),
 (23, 6.328590134088616),
 (24, 6.358836591863524),
 (25, 6.361560690159034),
 (26, 6.54300028051167),
 (27, 6.589976995151796),
 (28, 6.753853974306153),
 (29, 6.8129426111043605),
 (30, 6.8182534329036475),
 (31, 6.845093487377814),
 (32, 6.84758333252784),
 (33, 6.974163596563939),
 (34, 7.006450587141603),
 (35, 7.00811481401681),
 (36, 7.085005699655166),
 (37, 7.099091173491737),
 (38, 7.142359068758499),
 (39, 7.155829641275917

In [48]:
## arrival times for those that start OUT network but end INSIDE network (start,end]

## generate ALL t* under maxlambda in 24 hr period and then thin
maxlambda = np.max(dfoos.startoutendin_lambda)


#generate first customer arrival
u1 = random.uniform(0,1)

lambda0 = dfoos.loc[dfoos.index == 0, 'startoutendin_lambda'].iloc[0]


cust1 = -1/lambda0*np.log(u1)

vtime = cust1


cust_total = []

cust_total.append(vtime)

while vtime < 24:
    u1 = random.uniform(0,1)
    vtime = vtime - (1/maxlambda)*np.log(u1)
    cust_total.append(vtime)
    if len(cust_total) > 1000: 
        break



cust_total.pop(-1) #pop inplace=True by default

len(cust_total)



cust_ftotal = []
customer_startoutendin = []

ncust = 1

#thin arrival times here
for x in range(len(cust_total)):
    u2 = random.uniform(0,1)
    tlambda = np.floor(cust_total[x])
    tlambda = dfoos.loc[dfoos.hour == tlambda,'startoutendin_lambda'].iloc[0]
    if u2 <= tlambda / maxlambda:
        cust_ftotal.append(cust_total[x])
        customer_startoutendin.append((ncust, cust_total[x]))
        ncust +=1

cust_ftotal = pd.DataFrame(cust_ftotal)
cust_ftotal.columns = ["arrival_time"]
cust_ftotal['arrhour'] = cust_ftotal.arrival_time.apply(lambda x: np.floor(x))

##print list
cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))
(cust_ftotal.groupby('arrhour').agg(vcount = ('arrhour','count'))).sum()

customer_startoutendin

23.60573922353933

1000

,vcount
arrhour,
0.0,3
1.0,3
3.0,1
4.0,1
5.0,6
6.0,8
7.0,11
8.0,18
9.0,12


vcount    303
dtype: int64

[(1, 0.5243216017629104),
 (2, 0.7691653080727758),
 (3, 0.7709980218828008),
 (4, 1.0686050179462911),
 (5, 1.388265101845545),
 (6, 1.8542872194012714),
 (7, 3.5993754801993374),
 (8, 4.422088647758822),
 (9, 5.051970869466127),
 (10, 5.096352262643248),
 (11, 5.126939044074145),
 (12, 5.260658866372052),
 (13, 5.344136360613251),
 (14, 5.527452480813426),
 (15, 6.035561187588591),
 (16, 6.072700232250021),
 (17, 6.231384800967192),
 (18, 6.2600286588309775),
 (19, 6.636726822149843),
 (20, 6.713090046539737),
 (21, 6.811924601201591),
 (22, 6.900511900904493),
 (23, 7.24358086604537),
 (24, 7.283980561327201),
 (25, 7.4247810527568845),
 (26, 7.440240186332866),
 (27, 7.5146898550644),
 (28, 7.65436367291981),
 (29, 7.654777224143383),
 (30, 7.655401106900245),
 (31, 7.770905380072497),
 (32, 7.938829132235535),
 (33, 7.999217727003522),
 (34, 8.036591084570047),
 (35, 8.105977243426077),
 (36, 8.158049935177068),
 (37, 8.289745769422906),
 (38, 8.332062968776258),
 (39, 8.385842810